In [1]:
import numpy as np
import auto_fruit_search as afs

pygame 2.5.0 (SDL 2.28.0, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
fruit_list, fruit_true_pos, aruco_true_pos = afs.read_true_map("M4_prac_map_full.txt")
print(fruit_list)

#write to search_list.txt first to specify the order in which you want to search the list. 
search_list = afs.read_search_list()
print(search_list)

afs.print_target_fruits_pos(search_list, fruit_list, fruit_true_pos)

['garlic', 'capsicum', 'lime', 'lemon', 'orange', 'orange', 'pumpkin', 'potato', 'potato', 'tomato']
['garlic', 'lemon', 'lime', 'tomato', 'pumpkin']
Search order:
1) garlic at [0.7, -0.6]
2) lemon at [-0.4, -1.2]
3) lime at [-1.1, -0.1]
4) tomato at [0.8, 0.8]
5) pumpkin at [-0.6, 1.2]


In [3]:
# main loop
if __name__ == "__main__":
    parser = argparse.ArgumentParser("Fruit searching")
    parser.add_argument("--map", type=str, default='M4_true_map_full.txt') # change to 'M4_true_map_part.txt' for lv2&3
    parser.add_argument("--ip", metavar='', type=str, default='192.168.50.1')
    parser.add_argument("--port", metavar='', type=int, default=8080)
    args, _ = parser.parse_known_args()

    #copy over from operate, did not copy save/play data
    #parser.add_argument("--yolo_model", default='YOLO/model/yolov8_model.pt') #use for level3
    parser.add_argument("--calib_dir", type=str, default="calibration/param/")

    ppi = PenguinPi(args.ip,args.port)

    # read in the true map
    fruits_list, fruits_true_pos, aruco_true_pos = read_true_map(args.map)
    search_list = read_search_list()
    print_target_fruits_pos(search_list, fruits_list, fruits_true_pos)

    #currently no visuals added (see pygame in operate.py)
    offset=[0.1,0.1]
    endpos= fruits_true_pos[0] -offset #will need to iterate through,
    obstacles= np.concatenate((fruits_true_pos,aruco_true_pos),axis=None) #need to check output
    n_iter=100 #make sure not too long
    radius=0.2 #for clearance of obsticals
    stepSize= 0.15 #need large stepsize
    robot_pose = [0.0,0.0,0.0]

    counter=0
    # The following is only a skeleton code for semi-auto navigation
    while True:
        # enter the waypoints
        # instead of manually enter waypoints, you can give coordinates by clicking on a map, see camera_calibration.py from M2
        x,y = 0.0,0.0
        x = input("X coordinate of the waypoint: ")
        try:
            x = float(x)
        except ValueError:
            print("Please enter a number.")
            continue
        y = input("Y coordinate of the waypoint: ")
        try:
            y = float(y)
        except ValueError:
            print("Please enter a number.")
            continue

        # estimate the robot's pose
        robot_pose = get_robot_pose()

        #add pathplanning
        startpos = robot_pose
        endpos = [x,y]-offset

        rrt_star = rrt.RRT_star(startpos, endpos, obstacles, n_iter, radius, stepSize)
        # G = RRT(startpos, endpos, obstacles, n_iter, radius, stepSize)

        if rrt_star.success:
            path = rrt.dijkstra(G)
            print(path)
            plt.plot(G, obstacles, radius, path)
            for i in range(len(path)):
                # robot drives to the waypoint
                waypoint = path[i]
                drive_to_point(waypoint,robot_pose)
                print("Finished driving to waypoint: {}; New robot pose: {}".format(waypoint,robot_pose))              
        else:
            plt.plot(G, obstacles, radius)


        #after reaching endpoint should confirm target with YOLO
        #operate.Operate.detect_target() #only used w YOLO
        #self.detector_output= list of lists, box info [label,[x,y,width,height]] for all detected targets in image
        #once detect, no need to append to map, even if low certainty, so that path-planning will avoid it
        #add to fruit list, friut true pos (used in planning)

        # exit
        ppi.set_velocity([0, 0])
        uInput = input("Add a new waypoint? [Y/N]")
        if uInput == 'N':
            break


NameError: name 'argparse' is not defined